In [11]:
import boto3
import botocore
import mxnet as mx
from PIL import Image
import numpy as np
import math
import os

In [50]:
# Helper to get number of lines in file
# Returns number of lines in a file

def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    f.close()
    return i + 1


# Helper to parse line of lst
# Returns: 
# path to save image locally
# path of image in s3 to download image
# the labels and id assoc. with the image

def line_parser(line):
    
    s3_img_path = line[len(line) - 1][2:].strip()
    local_img_path = s3_img_path.split('/')
    local_img_path = local_img_path[len(local_img_path) - 1]
    
    img_label = line[1:len(line)-1]
    img_label = img_label[0].split(',')
    img_label = [float(i) for i in img_label]
    img_id = int(line[0]) 
    
    return local_img_path, s3_img_path, img_label, img_id

# Helper to calculate the resize dimentions of an image
# Parameters: 
# ndarray of image
# maximum width & height of resized image
# Returns:
# image array with proper size

def build_resized_image_array(img, max_width, max_height):
    width, height = img.size    
    resize_ratio = min(max_width/width, max_height/height)
    try:
        print("Worked <---")
        img_array = np.array(img.resize((int(width*resize_ratio),  int(height*resize_ratio))))
    except:
        print("Didn't work")
        return np.array([])
    else:
        return img_array

def build_img_header(img_label, img_id):
    return mx.recordio.IRHeader(0, img_label, img_id, 0)

def build_record(img_header, img_array):
    return mx.recordio.pack_img(img_header, img_array)

def download_image(bucket, s3_img_path, local_img_path):
    s3.Bucket(bucket).download_file(s3_img_path, local_img_path)
    
def remove_image(local_img_path):
    os.remove(local_img_path)
    
def get_num_lines_for_train_val_lsts(complete_set_lst, p_train, p_val):
    num_lines = file_len(complete_set_lst)
    train_num_lines = int(num_lines*p_train)
    val_num_lines = int(num_lines*p_val)
    return train_num_lines, val_num_lines


In [17]:
# x, y = get_num_lines_for_train_val_lsts('whole.lst', .90, .05)
# lst = open('whole.lst')
# t_lst = open('train.lst', 'w')
# v_lst = open('val.lst', 'w')
# test_lst = open('test.lst', 'w')


# for i, line in enumerate(lst):
#     if i <= x:
#         t_lst.write(line)
#     if i > x and i <= x+y:
#         v_lst.write(line)
#     else:
#         test_lst.write(line)

In [9]:
lst = open('train.lst')
s3 = boto3.resource('s3')

# save a copy of the previous version
if os.path.exists('train.rec'):
    os.rename('train.rec', 'train-OLD-COPY.rec')

bucket = 'pixelscrapper-user-content'
image_dir = 'images/'
train_rec = mx.recordio.MXRecordIO('train.rec', 'w')

train_rec.reset()


# --------------Debug resize func----------------------

# def build_resized_image_array(img, max_width, max_height):
#     width, height = img.size    
#     resize_ratio = min(max_width/width, max_height/height)
#     try:
#         img_array = np.array(img.resize((int(width*resize_ratio),  int(height*resize_ratio))))
#     except:
#         return np.array([])
#     else:
#         return img_array

# img = Image.open("stitch-template-034-graphic-element-embellishment-sew-sewing-thread-stitched-stitches-stitching.png")
# img = img.convert("RGBA")   
# img_array = build_resized_image_array(img, 224, 224)
# if img_array.size == 0:
#     img_header = mx.recordio.IRHeader(0, 1, 90, 0)
#     img_packed = mx.recordio.pack_img(img_header, img_array)
#     train_rec.write(img_packed)

# ------------------------------------

for line in lst:  
    line = line.split('\t')
    local_img_path, s3_img_path, img_label, img_id = line_parser(line)
    s3.Bucket(bucket).download_file(s3_img_path, local_img_path)
    
    img = Image.open(local_img_path)
    img = img.convert("RGBA")   
    img_array = build_resized_image_array(img, 224, 224)
    if img_array.size == 0 and:
        img_header = mx.recordio.IRHeader(0, img_label, img_id, 0)
        img_packed = mx.recordio.pack_img(img_header, img_array)
        train_rec.write(img_packed)
    
    os.remove(local_img_path)
    
train_rec.close()

error: OpenCV(4.2.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:877: error: (-215:Assertion failed) !image.empty() in function 'imencode'


In [10]:
%%bash
ls

Image Processing.ipynb
OLDtest.txt
stitch-template-034-graphic-element-embellishment-sew-sewing-thread-stitched-stitches-stitching.png
train.lst
train-OLD-COPY.rec
train.rec
Untitled.ipynb
val.lst
whole.lst
